# Multi-turn датасеты для обучения LLM

Этот notebook показывает как генерировать **multi-turn диалоги** для обучения LLM.

**4 типа диалогов:**
- **chain** — последовательные задачи (результат предыдущей используется в следующей)
- **followup** — уточняющие вопросы ("почему?", "как ещё можно?")
- **variations** — вариации задачи ("а если удвоить параметр?")
- **correction** — исправление ошибок (для RLHF)

**Формат выхода:**
```python
{
    "messages": [
        {"role": "system", "content": "..."},
        {"role": "user", "content": "Вычислите: 5 + 3"},
        {"role": "assistant", "content": "<think>...</think><answer>8</answer>"},
        {"role": "user", "content": "Теперь умножьте на 2"},
        {"role": "assistant", "content": "<think>...</think><answer>16</answer>"},
        ...
    ],
    "metadata": {"mode": "chain", "task_type": "arithmetic", ...}
}
```

In [ ]:
from re_rl import DatasetGenerator, MultiturnGenerator
from re_rl.multiturn_generator import MultiturnDialogue
import json

## 1. Chain — Последовательные задачи

Результат каждой задачи используется в следующей. Отлично подходит для обучения модели следить за контекстом диалога.

In [ ]:
gen = MultiturnGenerator()

# Генерируем один chain-диалог
chain = gen.generate_chain_dialogue(
    task_type="arithmetic",
    turns=4,  # 4 последовательных задачи
    language="ru",
    difficulty=3,
    reasoning_mode=True,
)

print("="*60)
print("CHAIN: Последовательные задачи")
print("="*60)
print(f"Количество turns: {chain.num_turns}\n")

for msg in chain.messages:
    role = msg['role'].upper()
    print(f"[{role}]")
    print(msg['content'])
    print()

In [ ]:
# Chain с уравнениями
chain_eq = gen.generate_chain_dialogue(
    task_type="linear",
    turns=3,
    language="ru",
    difficulty=5,
    reasoning_mode=True,
)

print("="*60)
print("CHAIN: Уравнения с подстановкой")
print("="*60)

for msg in chain_eq.messages:
    role = msg['role'].upper()
    print(f"[{role}]")
    print(msg['content'])
    print()

In [ ]:
# Chain с физикой (кинематика)
chain_phys = gen.generate_chain_dialogue(
    task_type="kinematics",
    turns=3,
    language="ru",
    difficulty=5,
    reasoning_mode=True,
)

print("="*60)
print("CHAIN: Физика (кинематика)")
print("="*60)

for msg in chain_phys.messages:
    role = msg['role'].upper()
    print(f"[{role}]")
    print(msg['content'])
    print()

## 2. Followup — Уточняющие вопросы

После решения задачи пользователь задаёт вопросы:
- "Почему ты использовал этот метод?"
- "Можно ли решить по-другому?"
- "Как проверить ответ?"

Учит модель объяснять своё решение.

In [ ]:
followup = gen.generate_followup_dialogue(
    task_type="quadratic",
    num_followups=3,  # 3 уточняющих вопроса после решения
    language="ru",
    difficulty=5,
    reasoning_mode=True,
)

print("="*60)
print("FOLLOWUP: Уточняющие вопросы")
print("="*60)
print(f"Количество turns: {followup.num_turns}\n")

for msg in followup.messages:
    role = msg['role'].upper()
    print(f"[{role}]")
    content = msg['content']
    # Обрезаем длинные ответы для читаемости
    if len(content) > 500:
        print(content[:500] + "...")
    else:
        print(content)
    print()

## 3. Variations — Вариации задачи

После решения задачи пользователь спрашивает:
- "А если удвоить параметр?"
- "А если значение будет отрицательным?"
- "Обобщи решение для произвольного X"

Учит модель обобщать и исследовать граничные случаи.

In [ ]:
variation = gen.generate_variation_dialogue(
    task_type="kinematics",
    num_variations=2,
    language="ru",
    difficulty=5,
    reasoning_mode=True,
)

print("="*60)
print("VARIATIONS: Вариации задачи")
print("="*60)
print(f"Количество turns: {variation.num_turns}\n")

for msg in variation.messages:
    role = msg['role'].upper()
    print(f"[{role}]")
    print(msg['content'])
    print()

## 4. Correction — Исправление ошибок (для RLHF)

Модель сначала даёт **неправильный ответ**, получает feedback и **исправляется**.

Очень полезно для RLHF — учит модель:
- Признавать ошибки
- Исправляться после feedback
- Перепроверять решения

In [ ]:
correction = gen.generate_correction_dialogue(
    task_type="arithmetic",
    language="ru",
    difficulty=4,
    reasoning_mode=True,
)

print("="*60)
print("CORRECTION: Исправление ошибок")
print("="*60)
print(f"Количество turns: {correction.num_turns}\n")

for msg in correction.messages:
    role = msg['role'].upper()
    print(f"[{role}]")
    print(msg['content'])
    print()

In [ ]:
# Correction с квадратным уравнением
correction_eq = gen.generate_correction_dialogue(
    task_type="quadratic",
    language="ru",
    difficulty=5,
    reasoning_mode=True,
)

print("="*60)
print("CORRECTION: Квадратное уравнение")
print("="*60)

for msg in correction_eq.messages:
    role = msg['role'].upper()
    print(f"[{role}]")
    print(msg['content'])
    print()

## 5. Генерация датасетов через DatasetGenerator

Удобные методы для генерации больших датасетов.

In [ ]:
import os
import random
from datetime import datetime

OUTPUT_DIR = "datasets"
os.makedirs(OUTPUT_DIR, exist_ok=True)

ds_gen = DatasetGenerator(output_dir=OUTPUT_DIR)

In [ ]:
# Генерация смешанного multi-turn датасета
print("="*60)
print("ГЕНЕРАЦИЯ СМЕШАННОГО MULTI-TURN ДАТАСЕТА")
print("="*60)

multiturn_dataset = ds_gen.generate_multiturn_dataset(
    modes=["chain", "followup", "variations", "correction"],  # Все 4 типа
    task_types=["arithmetic", "linear", "quadratic", "kinematics"],
    num_samples=1000,
    language="ru",
    difficulties=list(range(1, 11)),
    reasoning_mode=True,
    turns=3,
)

print(f"\nСгенерировано: {len(multiturn_dataset)} диалогов")

# Статистика по типам
from collections import Counter
mode_counts = Counter(d['metadata']['mode'] for d in multiturn_dataset)
print(f"\nРаспределение по типам:")
for mode, count in sorted(mode_counts.items()):
    print(f"  {mode}: {count} диалогов")

In [ ]:
# Пример из датасета
print("="*60)
print("ПРИМЕР ИЗ ДАТАСЕТА")
print("="*60)

example = multiturn_dataset[0]
print(f"Mode: {example['metadata']['mode']}")
print(f"Task type: {example['metadata']['task_type']}")
print(f"Difficulty: {example['metadata']['difficulty']}")
print(f"\nMessages ({len(example['messages'])}):")

for i, msg in enumerate(example['messages']):
    role = msg['role'].upper()
    content = msg['content'][:150] + "..." if len(msg['content']) > 150 else msg['content']
    print(f"  {i+1}. [{role}]: {content}")

In [ ]:
# Сохранение датасета
def save_jsonl(data, filepath):
    with open(filepath, 'w', encoding='utf-8') as f:
        for item in data:
            f.write(json.dumps(item, ensure_ascii=False) + '\n')

random.shuffle(multiturn_dataset)
split_idx = int(len(multiturn_dataset) * 0.9)

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
train_file = f"{OUTPUT_DIR}/multiturn_train_{timestamp}.jsonl"
eval_file = f"{OUTPUT_DIR}/multiturn_eval_{timestamp}.jsonl"

save_jsonl(multiturn_dataset[:split_idx], train_file)
save_jsonl(multiturn_dataset[split_idx:], eval_file)

print("="*60)
print("MULTI-TURN ДАТАСЕТ СОХРАНЁН!")
print("="*60)
print(f"\nФайлы:")
print(f"  Train: {train_file} ({split_idx} диалогов)")
print(f"  Eval: {eval_file} ({len(multiturn_dataset) - split_idx} диалогов)")

## 6. Специализированные датасеты

Можно генерировать датасеты только одного типа.

In [ ]:
# Только Chain-диалоги (для обучения следить за контекстом)
chain_dataset = ds_gen.generate_chain_dataset(
    task_type="arithmetic",
    num_samples=500,
    turns=4,
    language="ru",
    reasoning_mode=True,
)

print(f"Chain датасет: {len(chain_dataset)} диалогов")
print(f"Среднее количество сообщений: {sum(len(d['messages']) for d in chain_dataset) / len(chain_dataset):.1f}")

In [ ]:
# Только Correction-диалоги (для RLHF)
correction_dataset = ds_gen.generate_correction_dataset(
    task_type="arithmetic",
    num_samples=500,
    language="ru",
    reasoning_mode=True,
)

print(f"Correction датасет: {len(correction_dataset)} диалогов")
print(f"\nПример (модель делает ошибку и исправляется):")

ex = correction_dataset[0]
for msg in ex['messages']:
    role = msg['role'].upper()
    print(f"[{role}]: {msg['content'][:200]}{'...' if len(msg['content']) > 200 else ''}")
    print()

## 7. Формат данных для обучения

Формат совместим с:
- **Axolotl** (sharegpt format)
- **HuggingFace TRL** (chat format)
- **OpenAI fine-tuning API**

In [ ]:
# Структура одного примера
print("="*60)
print("СТРУКТУРА ДАННЫХ")
print("="*60)

example = multiturn_dataset[0]
print(json.dumps(example, ensure_ascii=False, indent=2)[:2000])

In [ ]:
# Конвертация в ShareGPT формат (для Axolotl)
def to_sharegpt(dialogue):
    """Конвертирует в ShareGPT формат для Axolotl."""
    conversations = []
    for msg in dialogue['messages']:
        if msg['role'] == 'system':
            conversations.append({"from": "system", "value": msg['content']})
        elif msg['role'] == 'user':
            conversations.append({"from": "human", "value": msg['content']})
        elif msg['role'] == 'assistant':
            conversations.append({"from": "gpt", "value": msg['content']})
    return {"conversations": conversations}

sharegpt_example = to_sharegpt(multiturn_dataset[0])
print("ShareGPT формат:")
print(json.dumps(sharegpt_example, ensure_ascii=False, indent=2)[:1500])

## 8. Английская версия

Все промпты поддерживают английский язык.

In [ ]:
# English chain dialogue
chain_en = gen.generate_chain_dialogue(
    task_type="arithmetic",
    turns=3,
    language="en",
    difficulty=3,
    reasoning_mode=True,
)

print("="*60)
print("ENGLISH CHAIN DIALOGUE")
print("="*60)

for msg in chain_en.messages:
    role = msg['role'].upper()
    print(f"[{role}]")
    print(msg['content'])
    print()

In [ ]:
# English correction dialogue
correction_en = gen.generate_correction_dialogue(
    task_type="arithmetic",
    language="en",
    difficulty=4,
    reasoning_mode=True,
)

print("="*60)
print("ENGLISH CORRECTION DIALOGUE")
print("="*60)

for msg in correction_en.messages:
    role = msg['role'].upper()
    print(f"[{role}]")
    print(msg['content'])
    print()

## Резюме

**Multi-turn генерация:**

| Метод | Описание | Применение |
|-------|----------|------------|
| `generate_chain_dataset()` | Последовательные задачи | Обучение следить за контекстом |
| `generate_followup_dataset()` | Уточняющие вопросы | Обучение объяснять решения |
| `generate_variation_dataset()` | Вариации параметров | Обучение обобщать |
| `generate_correction_dataset()` | Исправление ошибок | RLHF, самокоррекция |
| `generate_multiturn_dataset()` | Смешанный датасет | Универсальное обучение |

**Поддерживаемые типы задач для multi-turn:**
- `arithmetic` — арифметика
- `linear` — линейные уравнения
- `quadratic` — квадратные уравнения
- `kinematics` — кинематика
- `dynamics` — динамика